# Lazada Project: Extract

This part does extraction of ETL (Extract, Transform and Load).
Data from API is processed and saved into csv file.

In [83]:
#import libraries
import requests
import pandas as pd
import json

## Retrieving data from API

In [90]:
url = "https://lazada-api.p.rapidapi.com/lazada/search/items"

In [91]:
querystring = {"keywords":"airpods","site":"ph","page":"1","sort":"pop"}

In [92]:
headers = {
	"x-rapidapi-key": "", #personal api key deleted
	"x-rapidapi-host": "lazada-api.p.rapidapi.com"
}

In [93]:
response = requests.get(url, headers=headers, params=querystring)

In [108]:
#validate if response == 200
print(response.status_code)

200


In [95]:
data = response.json()

In [107]:
#save into json file
with open('lazada.json', 'w', encoding='utf-8') as outfile:
    json.dump(data, outfile, indent=4)

## Retrieving data from json file

In [135]:
with open('lazada.json', 'r', encoding='utf-8') as f:
    data = json.load(f)
#skip if data is retrieved from API

## Process json data

In [136]:
items = data['data']['items']

In [137]:
df = pd.DataFrame(items)

In [138]:
#normalize price info
df_price_info = pd.json_normalize(df['price_info'])
df_price_info.columns = ['sale_price','origin_price']
df = pd.concat([df,df_price_info],axis=1)
df = df.drop('price_info',axis=1)

In [139]:
#normalize review info
df_review_info = pd.json_normalize(df['review_info'])
df_review_info.columns = ['average_score','review_count']
df = pd.concat([df,df_review_info],axis=1)
df = df.drop('review_info',axis=1)

In [140]:
#normalize shop info
df_shop_info = pd.json_normalize(df['shop_info'])
df = pd.concat([df,df_shop_info],axis=1)
df = df.drop('shop_info',axis=1)

In [141]:
#normalize delivery info
df_delivery_info = pd.json_normalize(df['delivery_info'])
df = pd.concat([df,df_delivery_info],axis=1)
df = df.drop('delivery_info',axis=1)

In [142]:
df.to_csv('lazada.csv')